# Student Details

***

**Name:** Hoai Nhan Nguyen <br>
**Student Number:** sba24098 <br>
**Course:** Higher Diploma in Science in Artificial Intelligence Applications 

***

# Assessment Task

Students are advised to review and adhere to the submission requirements documented after the assessment task.

# Assessment details

You are required to use the dataset attached “mye22final.xlsx”, that contains data on Estimates of the population for the UK, England, Wales, Scotland, and Northern Ireland for the years 2011 and 2022.

This Data is Licenced under open-government-licence v3.

https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/

Create an interactive Dashboard aimed at younger adults (18 - 35 years).

The dashboard may be created using ANY pythonic method within a Jupyter Notebook. PYTHON FILES May be used after they have been generated from the Jupyter Notebook or as helper files, but the main work MUST be completed in Jupyter Notebook. (You may (if you wish) Host the Dashboard using STREAMLIT or Similar). The Dashboard may be viewed in HTML format, within the notebook or as a standalone application based on the techniques covered in the class, (Tkinter, panel, dash, Jupyter-dash etc…). All final code and files MUST be uploaded to MOODLE and progress shown in your GitHub Classroom commits.

<b>YOU MAY NOT USE POWERBI, TABLEAU, LOOKER, or any other generalized tool for this assignment.</b>

The Dashboard must detail the following requirements:

* Be Interactive and include Hover functionality.
* Contain at least 2 Visualizations.
* Display population densities by Geographic location.
* Allow the user to select and display population densities by age AND by Gender.
* Display a comparison between the 2011 figures and the 2022 figures.

(70 Marks)

Discuss in detail your rationale and justification for all stages of data preparation for your visualizations and how your dashboard is designed with this demographic (younger adults (18 - 35 years)) in mind.  

(30 marks)
***